In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import json
import os
import re
from typing import Tuple, Dict, Any, List

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from transformers import GPT2Model, AutoTokenizer, HubertModel, AutoFeatureExtractor
from huggingface_hub import notebook_login
from datasets import load_dataset

In [3]:
notebook_login()

In [5]:
dataset = load_dataset('danielroncel/dstc2_audios')

Generating train split:   0%|          | 0/25516 [00:00<?, ? examples/s]

In [6]:
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/hubert-base-ls960")

preprocessor_config.json:   0%|          | 0.00/213 [00:00<?, ?B/s]

In [7]:
def repeat_audio(original_audio, target_duration):

    if len(original_audio) >= target_duration:
      return original_audio

    audio = original_audio.copy()
    while len(audio) < target_duration:
        audio = np.concatenate([audio, original_audio], axis=0)
    return audio

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    paths = [x["path"] for x in examples["audio"]]

    audio_arrays = [repeat_audio(audio, feature_extractor.sampling_rate * 3.0) for audio in audio_arrays]

    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        padding='max_length',
        max_length=int(feature_extractor.sampling_rate * 3.0),
        truncation=True,
    )

    inputs = inputs['input_values']

    return {'path': paths, 'input_values': inputs}

In [8]:
encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset

Map:   0%|          | 0/25516 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'session_ids', 'path', 'input_values'],
        num_rows: 25516
    })
})

In [9]:
encoded_dataset = encoded_dataset.remove_columns(["audio"])
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['session_ids', 'path', 'input_values'],
        num_rows: 25516
    })
})

In [11]:
encoded_dataset.push_to_hub('danielroncel/dstc2_audios_input_hubert')

Uploading the dataset shards:   0%|          | 0/10 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/danielroncel/dstc2_audios_input_hubert/commit/ba0e6eb37bb1cade9b5b04abc2fc6075b3fa6a97', commit_message='Upload dataset', commit_description='', oid='ba0e6eb37bb1cade9b5b04abc2fc6075b3fa6a97', pr_url=None, pr_revision=None, pr_num=None)